In [0]:
DECLARE InsertStr STRING;
DECLARE DeleteStr STRING;
DECLARE ColumnStr STRING;

SET VAR DeleteStr  = "  Delete from apachedataingestion.nr.widetable_5g_du_dy_gold2 where CalendarDate = date_format(date_add(now()," || :days || "),'yyyy-MM-dd')";

EXECUTE IMMEDIATE DeleteStr;

SET VAR ColumnStr = "MAX(S5NR_MaxRRCconnectedUsersPerDU)	S5NR_MaxRRCconnectedUsersPerDU,
					SUM(EndcAddAtt_DU)	EndcAddAtt_DU,
					SUM(S5NR_AvgRRCconnectedUsersPerDU_Num)	S5NR_AvgRRCconnectedUsersPerDU_Num,
					SUM(S5NR_AvgRRCconnectedUsersPerDU_Denom)	S5NR_AvgRRCconnectedUsersPerDU_Denom,
					SUM(S5NR_F1UPacketLossRate_DL_PCT_Num)	S5NR_F1UPacketLossRate_DL_PCT_Num,
					SUM(S5NR_F1UPacketLossRate_DL_PCT_Denom)	S5NR_F1UPacketLossRate_DL_PCT_Denom,
					SUM(S5ND_F1UPacketLossRate_DL_PCT_Num)	S5ND_F1UPacketLossRate_DL_PCT_Num,
					SUM(S5ND_F1UPacketLossRate_DL_PCT_Denom)	S5ND_F1UPacketLossRate_DL_PCT_Denom,
					SUM(S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Num)	S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Num,
					SUM(S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Denom)	S5ND_F1U_PktLoss_DL_SA_ppm_PCT_Denom,
					SUM(S5ND_AvgRRCUsers_SANSA_perDU_Num)	S5ND_AvgRRCUsers_SANSA_perDU_Num,
					SUM(S5ND_AvgRRCUsers_SANSA_perDU_Denom)	S5ND_AvgRRCUsers_SANSA_perDU_Denom,
					MAX(S5ND_MaxRRCUsers_SANSA_perDU)	S5ND_MaxRRCUsers_SANSA_perDU,
					SUM(F1U_DELAY_NSA_Num)	F1U_DELAY_NSA_Num,
					SUM(F1U_DELAY_NSA_Denom)	F1U_DELAY_NSA_Denom,
					SUM(F1_UL_packet_loss_Num)	F1_UL_packet_loss_Num,
					SUM(F1_UL_packet_loss_Denom)	F1_UL_packet_loss_Denom
					";

SET VAR InsertStr  = "
                      Insert into apachedataingestion.nr.widetable_5g_du_dy_gold2
                      with v_widetable_5g_du_hr_gold2 as 
					  (
                        select * from apachedataingestion.nr.widetable_5g_du_hr_gold2 
						where CalendarDate = date_format(date_add(now(), " || :days ||"),'yyyy-MM-dd')  
                      )
                      select 
                        '24hr' MW_BH,
                        CalendarDate,
						GNODEB_ID,
						du_id,
						MARKET_NAME,
						SUB_MARKET_ID,
						SUB_MARKET_NAME,
						NR_TYPE,
                        "|| ColumnStr ||"
						from  v_widetable_5g_du_hr_gold2
						group by 
						CalendarDate,
						  GNODEB_ID,
						  du_id,
						  MARKET_NAME,
						  NR_TYPE,
						  SUB_MARKET_ID,
						  SUB_MARKET_NAME
                union all
                      select 
                        'MW_Excluded' MW_BH,
                        CalendarDate,
						GNODEB_ID,
						du_id,
						MARKET_NAME,
						SUB_MARKET_ID,
						SUB_MARKET_NAME,
						NR_TYPE,
                        "|| ColumnStr ||"
						from  v_widetable_5g_du_hr_gold2
						where Hour24 >= 6
						group by 
						CalendarDate,
						  GNODEB_ID,
						  du_id,
						  MARKET_NAME,
						  NR_TYPE,
						  SUB_MARKET_ID,
						  SUB_MARKET_NAME
                union all
                      select 
                        'FBH_20' MW_BH,
                        CalendarDate,
						GNODEB_ID,
						du_id,
						MARKET_NAME,
						SUB_MARKET_ID,
						SUB_MARKET_NAME,
						NR_TYPE,
                        "|| ColumnStr ||"
						from  v_widetable_5g_du_hr_gold2
						where Hour24 = 20
						group by 
						CalendarDate,
						  GNODEB_ID,
						  du_id,
						  MARKET_NAME,
						  NR_TYPE,
						  SUB_MARKET_ID,
						  SUB_MARKET_NAME
                union all
                      select 
                        'FBH_15' MW_BH,
                        CalendarDate,
						GNODEB_ID,
						du_id,
						MARKET_NAME,
						SUB_MARKET_ID,
						SUB_MARKET_NAME,
						NR_TYPE,
                        "|| ColumnStr ||"
						from  v_widetable_5g_du_hr_gold2
						where Hour24 = 15
						group by 
						CalendarDate,
						  GNODEB_ID,
						  du_id,
						  MARKET_NAME,
						  NR_TYPE,
						  SUB_MARKET_ID,
						  SUB_MARKET_NAME
                              
                      ";


EXECUTE IMMEDIATE InsertStr;